In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
import re
from datetime import datetime

In [84]:
data=pd.read_csv('listings_mad.csv')

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17634 entries, 0 to 17633
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            17634 non-null  int64  
 1   listing_url                                   17634 non-null  object 
 2   scrape_id                                     17634 non-null  int64  
 3   last_scraped                                  17634 non-null  object 
 4   name                                          17631 non-null  object 
 5   description                                   16814 non-null  object 
 6   neighborhood_overview                         10403 non-null  object 
 7   picture_url                                   17633 non-null  object 
 8   host_id                                       17634 non-null  int64  
 9   host_url                                      17634 non-null 

In [86]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

name                               3
description                      820
neighborhood_overview           7231
picture_url                        1
host_name                          7
host_since                         7
host_location                     48
host_about                      8809
host_response_time              6292
host_response_rate              6292
host_acceptance_rate            6402
host_is_superhost                  7
host_thumbnail_url                 7
host_picture_url                   7
host_neighbourhood              6591
host_listings_count                7
host_total_listings_count          7
host_has_profile_pic               7
host_identity_verified             7
neighbourhood                   7231
bathrooms                      17634
bathrooms_text                    29
bedrooms                        1339
beds                             843
minimum_minimum_nights             1
maximum_minimum_nights             1
minimum_maximum_nights             1
m

In [87]:
# drop these variables: Because there are too many missing rule of thumb: delete if more than 7000 is missing
data.drop(columns=['host_about','bathrooms', 'calendar_updated', 'license', 
                  'neighborhood_overview',  'neighbourhood'], inplace=True)


In [88]:
# process the missing value
# fill mean strategy with these attributes:
data['host_listings_count']=data['host_listings_count'].fillna(np.nanmean(data['host_listings_count']))
data['host_total_listings_count']=data['host_total_listings_count'].fillna(np.nanmean(data['host_total_listings_count']))
data['bedrooms']=data['bedrooms'].fillna(np.nanmean(data['bedrooms']))
data['beds']=data['beds'].fillna(np.nanmean(data['beds']))

In [89]:
# drop rows if no value on these variables
data.dropna(subset=['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                    'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                    'review_scores_value', 'reviews_per_month', 'bathrooms_text'], inplace=True)

In [90]:
#dummies
data['host_is_superhost']=(data['host_is_superhost']=='t').astype(int)
data['host_has_profile_pic']=(data['host_has_profile_pic']=='t').astype(int)
data['host_identity_verified']=(data['host_identity_verified']=='t').astype(int)
data['has_availability']=(data['has_availability']=='t').astype(int)
data['instant_bookable']=(data['instant_bookable']=='t').astype(int)

In [91]:
# Because the missing number of these attributes is small, they are assigned randomly
data['host_is_superhost']=data['host_is_superhost'].fillna('t')
data['host_has_profile_pic']=data['host_has_profile_pic'].fillna('t')
data['host_identity_verified']=data['host_identity_verified'].fillna('t')

In [92]:
data.property_type.value_counts()

Entire rental unit                     7389
Private room in rental unit            3253
Entire loft                             515
Private room in residential home        357
Entire condominium (condo)              342
Private room in condominium (condo)     219
Entire serviced apartment               195
Private room in hostel                  127
Entire residential home                 117
Shared room in rental unit               73
Private room in casa particular          61
Private room in bed and breakfast        60
Room in boutique hotel                   48
Room in hostel                           47
Private room in guest suite              36
Private room in townhouse                35
Room in hotel                            34
Shared room in hostel                    31
Private room in loft                     30
Entire guest suite                       27
Private room in guesthouse               26
Private room in serviced apartment       25
Private room in chalet          

In [93]:
# Rename roomt type because it is too long, just two classes
data= data.loc[data.property_type.isin(['Entire rental unit', 'Private room in rental unit'])]

In [94]:
data.property_type = ['entire uint' if x == 'Entire rental unit' else 'private room' for x in data.property_type]
data['f_property_type'] = data['property_type'].astype('category')

In [95]:
data.room_type.value_counts()

Entire home/apt    7389
Private room       3253
Name: room_type, dtype: int64

In [96]:
data['f_room_type'] = data['room_type'].astype('category')
data["f_room_type2"] = (
    data["f_room_type"]
    .replace(
        {
            "Entire home/apt": "Entire/Apt",
            "Private room": "Private",
        }
    )
    .astype("category")
)

In [97]:
data['f_neighbourhood_cleansed'] = data['neighbourhood_cleansed'].astype('category')
data['f_neighbourhood_group_cleansed'] = data['neighbourhood_group_cleansed'].astype('category')

In [98]:
data['n_days_since'] = pd.to_datetime(
    data['calendar_last_scraped'], format='%Y-%m-%d'
) - pd.to_datetime(data["first_review"], format="%Y-%m-%d")
# Previous row creates a timedelta object in each row. Get the elapsed number of days like:
data['n_days_since'] = [x.days for x in data['n_days_since']]

In [100]:
for column in (
    'accommodates',
    'beds',
    'minimum_nights',
    'number_of_reviews',
    'review_scores_rating',
    'reviews_per_month'
):

    data["n_" + column] = pd.to_numeric(data[column], errors="coerce")


In [101]:
data['accommodates']

0        2
1        1
3        3
5        4
8        4
        ..
17450    1
17454    2
17467    2
17495    2
17554    4
Name: accommodates, Length: 10642, dtype: int64

In [102]:
# keep columns if contain d_, n_,f_, p_, usd_ and some others
data = data.filter(regex=('^d_.*|^n_.*|^f_.*')).join(
    data[
        [
            'price',
            'id',
            'neighbourhood_cleansed',
            'room_type',
            'property_type',
        ]
    ]
)

In [103]:
data['price']

0        $60.00
1        $31.00
3        $92.00
5        $85.00
8        $81.00
          ...  
17450    $21.00
17454    $80.00
17467    $50.00
17495    $65.00
17554    $40.00
Name: price, Length: 10642, dtype: object

In [104]:
data['price2'] = data['price'].str.replace('$', '',regex=True)

In [105]:
data['price2'] = data['price2'].str.replace('.', '',regex=True)

In [106]:
data['price2'] = data['price2'].str.replace(',', '',regex=True)

In [107]:
data['price2']=data['price2'].astype(float)
data['price2']=data['price2'].astype(int)

In [110]:
data['price2']=data['price2']/100

In [111]:
data['price2']

0        60.0
1        31.0
3        92.0
5        85.0
8        81.0
         ... 
17450    21.0
17454    80.0
17467    50.0
17495    65.0
17554    40.0
Name: price2, Length: 10642, dtype: float64

In [112]:
data.n_accommodates

0        2
1        1
3        3
5        4
8        4
        ..
17450    1
17454    2
17467    2
17495    2
17554    4
Name: n_accommodates, Length: 10642, dtype: int64

In [113]:
# Squares and further values to create
data = data.assign(
    n_accommodates2=lambda x: x["n_accommodates"] ** 2,
    ln_accommodates=lambda x: np.log(x["n_accommodates"]),
    ln_accommodates2=lambda x: np.log(x["n_accommodates"]) ** 2,
    ln_beds=lambda x: np.log(x["n_beds"]),
    ln_number_of_reviews=lambda x: np.log(x["n_number_of_reviews"] + 1),
)

In [116]:
data['price2'].describe()

count    10642.000000
mean       109.731254
std        361.755599
min          7.000000
25%         42.000000
50%         70.000000
75%        106.000000
max       9999.000000
Name: price2, dtype: float64

In [118]:
#drop top 25 percentile
data = data.loc[lambda x: x.price2 < 106]

In [119]:
# Pool num of reviews to 3 categories: none, 1-51 and >51

bins = pd.IntervalIndex.from_tuples(
    [(0, 1), (1, 51), (51, max(data.n_number_of_reviews))], closed="left"
)
fnor = pd.cut(data["n_number_of_reviews"].to_list(), bins, labels=["0", "1", "2"])
fnor.categories = [0, 1, 2]
data["f_number_of_reviews"] = fnor
data["f_number_of_reviews"].value_counts(dropna=False)

<ipython-input-119-9d8ed426229e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["f_number_of_reviews"] = fnor


1      5546
2      2406
NaN       1
0         0
Name: f_number_of_reviews, dtype: int64

In [120]:
# Pool and categorize the number of minimum nights: 1,2,3, 3+

bins = pd.IntervalIndex.from_tuples(
    [(1, 2), (2, 3), (3, max(data.n_minimum_nights))], closed="left"
)
f_min_n = pd.cut(data["n_minimum_nights"].to_list(), bins, labels=["1", "2", "3"])
f_min_n.categories = [1, 2, 3]
data["f_minimum_nights"] = f_min_n
data["f_minimum_nights"].value_counts(dropna=False)

<ipython-input-120-fab1080d13f5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["f_minimum_nights"] = f_min_n


3      3088
1      2544
2      2319
NaN       2
Name: f_minimum_nights, dtype: int64

In [121]:
# Change Infinite values with NaNs
data = data.replace([np.inf, -np.inf], np.nan)

In [122]:
# what to do with missing values?
# 1. drop if no target
data = data.loc[lambda x: x.price2.notnull()]

In [124]:
# 2. imput when few, not that important
data = data.assign(
    n_beds=lambda x: np.where(x["n_beds"].isnull(), x["n_accommodates"], x["n_beds"]),
    f_minimum_nights=lambda x: x["f_minimum_nights"].fillna(1),
    f_number_of_reviews=lambda x: x["f_number_of_reviews"].fillna(1),
    ln_beds=lambda x: x["ln_beds"].fillna(0),
)

In [126]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

Series([], dtype: int64)

In [127]:
# 4. Replace missing variables re reviews with zero, when no review + add flags
data = data.assign(
    flag_days_since=np.multiply(data.n_days_since.isna(), 1),
    n_days_since=data.n_days_since.fillna(np.median(data.n_days_since.dropna())),
    flag_review_scores_rating=np.multiply(data.n_review_scores_rating.isna(), 1),
    n_review_scores_rating=data.n_review_scores_rating.fillna(
        np.median(data.n_review_scores_rating.dropna())
    ),
    flag_reviews_per_month=np.multiply(data.n_reviews_per_month.isna(), 1),
    n_reviews_per_month=data.n_reviews_per_month.fillna(
        np.median(data.n_reviews_per_month.dropna())
    ),
    flag_n_number_of_reviews=np.multiply(data.n_number_of_reviews.isna(), 1),
)

In [128]:
# redo features
# Create variables, measuring the time since: squared, cubic, logs
data = data.assign(
    ln_days_since=lambda x: np.log(x["n_days_since"] + 1),
    ln_days_since2=lambda x: np.log(x["n_days_since"] + 1) ** 2,
    ln_days_since3=lambda x: np.log(x["n_days_since"] + 1) ** 3,
    n_days_since2=lambda x: x["n_days_since"] ** 2,
    n_days_since3=lambda x: x["n_days_since"] ** 3,
    ln_review_scores_rating=lambda x: np.log(x["n_review_scores_rating"]),
)

/home/ubuntu/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [129]:
data.ln_days_since = data["ln_days_since"].fillna(0)
data.ln_days_since2 = data["ln_days_since2"].fillna(0)
data.ln_days_since3 = data["ln_days_since3"].fillna(0)

In [130]:
data.describe()

,n_days_since,n_accommodates,n_beds,n_minimum_nights,n_number_of_reviews,n_review_scores_rating,n_reviews_per_month,id,price2,n_accommodates2,...,flag_days_since,flag_review_scores_rating,flag_reviews_per_month,flag_n_number_of_reviews,ln_days_since,ln_days_since2,ln_days_since3,n_days_since2,n_days_since3,ln_review_scores_rating
count,7953.000000,7953.000000,7953.000000,7953.000000,7953.000000,7953.000000,7953.000000,7.953000e+03,7953.00000,7953.000000,...,7953.0,7953.0,7953.0,7953.0,7953.000000,7953.000000,7953.000000,7.953000e+03,7.953000e+03,7953.000000
mean,1158.175657,2.754558,1.713513,6.791651,52.718597,4.639622,1.417748,2.776392e+07,56.79580,9.659877,...,0.0,0.0,0.0,0.0,6.636270,45.463224,317.829916,1.937956e+06,3.856923e+09,-inf
std,772.437817,1.439634,0.942130,33.637092,80.945491,0.463599,1.611978,1.498861e+07,25.19783,10.665465,...,0.0,0.0,0.0,0.0,1.193029,13.652850,124.708870,2.215514e+06,6.425333e+09,NaN
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.010000,6.369000e+03,7.00000,1.000000,...,0.0,0.0,0.0,0.0,0.693147,0.480453,0.333025,1.000000e+00,1.000000e+00,-inf
25%,639.000000,2.000000,1.000000,1.000000,4.000000,4.500000,0.250000,1.637624e+07,35.00000,4.000000,...,0.0,0.0,0.0,0.0,6.461468,41.750571,269.769986,4.083210e+05,2.609171e+08,1.504077
50%,1079.000000,2.000000,1.000000,2.000000,19.000000,4.750000,0.850000,2.757156e+07,56.00000,4.000000,...,0.0,0.0,0.0,0.0,6.984716,48.786262,340.758201,1.164241e+06,1.256216e+09,1.558145
75%,1651.000000,4.000000,2.000000,3.000000,66.000000,4.920000,2.020000,4.028021e+07,77.00000,16.000000,...,0.0,0.0,0.0,0.0,7.409742,54.904276,406.826516,2.725801e+06,4.500297e+09,1.593309
max,3910.000000,16.000000,10.000000,1125.000000,758.000000,5.000000,24.590000,5.313910e+07,105.00000,256.000000,...,0.0,0.0,0.0,0.0,8.271548,68.418513,565.927036,1.528810e+07,5.977647e+10,1.609438


In [131]:
data.to_csv('listings_mad_cleaned.csv')